## Result 2: VSI
Vertical signal integrity (VSI) was accessed using `ovrl.py` (See `Ovrlpy_all_transcripts.ipynb`).  

This notebook focuses on the VSI map at Bregma = -0.24 and VSI differences between MOD1 and MOD2.


to do:  
<ul>
  <li><input type="checkbox"> add squares in Cell_VSI showing the ROIs</li>
</ul>

### data

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

_BIH_CMAP = LinearSegmentedColormap.from_list(
    "BIH",
    [
        "#430541",
        "mediumvioletred",
        "violet",
        "powderblue",
        "powderblue",
        "white",
        "white",
    ][::-1],
)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

heat_cmap = sns.color_palette("RdYlBu_r", as_cmap=True)
vsi_camp = sns.color_palette("ch:s=.25,rot=-.25", as_cmap=True)

#### Signals in the Tissue Section

In [ ]:
MERFISH_data_folder_path = Path("../../data/mouse_hypothalamus/MERFISH/")

In [ ]:
columns = [
    "Centroid_X",
    "Centroid_Y",
    "Centroid_Z",
    "Gene_name",
    "Cell_name",
    "Total_brightness",
    "Area",
    "Error_bit",
    "Error_direction",
]

signal_coordinate_df = pd.read_csv(
    MERFISH_data_folder_path / "merfish_barcodes_example.csv", usecols=columns
).rename(
    columns={
        "Centroid_X": "x",
        "Centroid_Y": "y",
        "Centroid_Z": "z",
        "Gene_name": "gene",
    }
)


# remove dummy molecules
signal_coordinate_df = signal_coordinate_df.loc[
    ~signal_coordinate_df["gene"].str.contains("Blank|NegControl"),
]

signal_coordinate_df["gene"] = signal_coordinate_df["gene"].astype("category")

# shift the coordinates to avoid the negative values
coordinate_x_m =  signal_coordinate_df['x'].min()
coordinate_y_m =  signal_coordinate_df['y'].min()
signal_coordinate_df['x'] = signal_coordinate_df['x'] - coordinate_x_m
signal_coordinate_df['y'] = signal_coordinate_df['y'] - coordinate_y_m

# make a copy to avoid SettingWithCopyWarning
signal_coordinate_df = signal_coordinate_df.copy()

#### Results of Ovrlpy

results:  
- signal integrity  
- signal strength  

In [ ]:
ovrlpy_result_folder = Path("../../data/results/VSI")

In [ ]:
signal_integrity = np.loadtxt(ovrlpy_result_folder/"SignalIntegrity.txt")
signal_strength = np.loadtxt(ovrlpy_result_folder/"SignalStrength.txt")

#### Results of BANKSY

In [ ]:
banksy_folder_path = Path("../../data/banksy_results/")

In [ ]:
columns = [
    "Centroid_X",
    "Centroid_Y",
    "Bregma",
    "lam0.2",
]

banksy_result = pd.read_csv(
    banksy_folder_path / 'banksy_cluster.txt', usecols=columns, sep = '\t'
).rename(
    columns={
        "Centroid_X": "x",
        "Centroid_Y": "y",
        "Bregma": "Bregma",
        "lam0.2": "banksy_cluster",
    }
)

banksy_result = banksy_result[banksy_result['Bregma'] == -0.24]

banksy_result['x'] = banksy_result['x'] - coordinate_x_m
banksy_result['y'] = banksy_result['y'] - coordinate_y_m

banksy_result = banksy_result.copy()

#### Segmentation Dataset

In [ ]:
merfish_data = pd.read_csv(
    MERFISH_data_folder_path / "merfish_all_cells.csv"
    ).rename(
    columns={
        "Centroid_X": "x",
        "Centroid_Y": "y"
    }
)

merfish_data = merfish_data.drop(columns=[col for col in merfish_data.columns if col == 'Fos' or col.startswith('Blank_')])
merfish_data = merfish_data[merfish_data["Cell_class"] != "Ambiguous"]
merfish_data = merfish_data[merfish_data['Animal_ID'] == 1]
merfish_data = merfish_data[merfish_data['Bregma'] == -0.24]

merfish_data['x'] = merfish_data['x'] - coordinate_x_m
merfish_data['y'] = merfish_data['y'] - coordinate_y_m

merfish_data['banksy'] = banksy_result['banksy_cluster'].values

merfish_data = merfish_data.copy()

In [ ]:
cell_class_m = {'Astrocyte': 'Astrocyte',
 'Endothelial 1': 'Endothelial',
 'Endothelial 2': 'Endothelial',
 'Endothelial 3': 'Endothelial',
 'Ependymal': 'Ependymal',
 'Excitatory': 'Excitatory',
 'Inhibitory': 'Inhibitory',
 'Microglia': 'Microglia',
 'OD Immature 1': 'OD Immature',
 'OD Immature 2': 'OD Immature',
 'OD Mature 1': 'OD Mature',
 'OD Mature 2': 'OD Mature',
 'OD Mature 3': 'OD Mature',
 'OD Mature 4': 'OD Mature',
 'Pericytes': 'Pericytes'}

merfish_data['Cell_class'] = merfish_data['Cell_class'].map(cell_class_m)
merfish_data = merfish_data.sort_values(by='Cell_class')

merfish_data = merfish_data.copy()

In [ ]:
# original cell types
merfish_024 = merfish_data[merfish_data['Bregma'] == -0.24]

class_024 = {'Astrocyte':1,
 'Endothelial':2,
 'Ependymal':3,
 'Excitatory':4,
 'Inhibitory':5,
 'Microglia':6,
 'OD Immature':7,
 'OD Mature':8,
 'Pericytes':9}

merfish_024['Cell_class'] = merfish_024['Cell_class'].map(class_024)

#### Cell Boundaries Dataset

In [ ]:
boundaries_df = pd.read_csv(MERFISH_data_folder_path/'cellboundaries_example_animal.csv')
boundaries_df = boundaries_df.dropna(subset=['boundaryX', 'boundaryY'])

In [ ]:
cell_ids = merfish_data['Cell_ID']
boundaries_df = boundaries_df[boundaries_df['feature_uID'].isin(cell_ids)]
boundaries_df = boundaries_df.merge(
    merfish_data[['Cell_ID', 'x', 'y', 'banksy']],
    left_on='feature_uID',
    right_on='Cell_ID',
    how='inner'
)
boundaries_df = boundaries_df.drop(columns=['Cell_ID'])

boundaries_df['boundaryX'] = boundaries_df['boundaryX'].apply(lambda x: [float(i) for i in x.split(';')] if isinstance(x, str) else x)
boundaries_df['boundaryY'] = boundaries_df['boundaryY'].apply(lambda x: [float(i) for i in x.split(';')] if isinstance(x, str) else x)

boundaries_df['boundaryX'] = boundaries_df['boundaryX'].apply(lambda x: [i - coordinate_x_m for i in x] if isinstance(x, list) else x)
boundaries_df['boundaryY'] = boundaries_df['boundaryY'].apply(lambda x: [i - coordinate_y_m for i in x] if isinstance(x, list) else x)

boundaries_df = boundaries_df.copy()

#### MOD
BANKSY cluster:  
- 7: MOD_gm / MOD2  
- 8: MOD_wm / MOD1

In [ ]:
# boundaries, MOD
# MOD_boundaries = boundaries_df[(boundaries_df['banksy'] == 8) | (boundaries_df['banksy'] == 7)]
MOD1_boundaries = boundaries_df[boundaries_df['banksy'] == 8]
MOD2_boundaries = boundaries_df[boundaries_df['banksy'] == 7]

### VSI across the slice
Main findings:  
- Lower VSI at cell borders
- May cause false cell identification

#### functions

In [ ]:
# extract_cell_integrity_within_boundary: extracts cell integrity and signal strength based on given boundaries.

from shapely.geometry import Polygon, Point

def extract_cell_integrity_within_boundary(
    boundary_df, 
    sig_integrity,
    signal_strength,
    integrity_size=1800
):

    # Initialize arrays to store cell integrity and strength
    cell_integrity = np.zeros((integrity_size, integrity_size))
    cell_strength = np.zeros((integrity_size, integrity_size))

    # Loop through each row in the boundary DataFrame
    for idx, row in boundary_df.iterrows():
        # Create the polygon from boundary coordinates
        polygon = Polygon(zip(row['boundaryX'], row['boundaryY']))
        
        # Determine the bounding box of the polygon
        x_min, x_max = int(np.floor(polygon.bounds[0])), int(np.ceil(polygon.bounds[2]))
        y_min, y_max = int(np.floor(polygon.bounds[1])), int(np.ceil(polygon.bounds[3]))
        
        # Ensure bounding box coordinates stay within the grid limits
        x_min, x_max = max(0, x_min), min(integrity_size, x_max)
        y_min, y_max = max(0, y_min), min(integrity_size, y_max)

        # Generate grid points within the bounding box
        y_indices, x_indices = np.meshgrid(range(y_min, y_max), range(x_min, x_max), indexing='ij')
        points = np.column_stack([x_indices.ravel(), y_indices.ravel()])
        
        # Identify points within the polygon or on its boundary
        # mask = np.array([polygon.contains(Point(x, y)) or polygon.touches(Point(x, y)) for x, y in points])
        epsilon = 0.7  # Small tolerance to handle floating-point precision issues
        mask = np.array([polygon.contains(Point(x, y)) or polygon.touches(Point(x, y)) or polygon.boundary.distance(Point(x, y)) < epsilon for x, y in points])
        mask = mask.reshape(y_indices.shape)

        # Extract the subgrid for integrity and strength within the bounding box
        subgrid_int = sig_integrity[y_min:y_max, x_min:x_max]
        subgrid_str = signal_strength[y_min:y_max, x_min:x_max]

        # Assign values to the cell arrays where the mask is True
        cell_integrity[y_min:y_max, x_min:x_max][mask] = subgrid_int[mask]
        cell_strength[y_min:y_max, x_min:x_max][mask] = subgrid_str[mask]

    # Return the computed cell integrity and strength grids
    return cell_integrity, cell_strength


In [ ]:
# plot_cell_integrity: plots the cell integrity heatmap with an optional histogram and optional boundarys.

def plot_cell_integrity(
    cell_integrity,
    cell_strength,
    signal_threshold=3.0,
    figure_height=10,
    cmap="BIH",
    plot_hist=True,
    boundary_df=None,
    plot_boundarys=False,
    marker_signals=None,
    plot_markers=False,
    boundary_color="yellow",
):
    
    # Validate inputs
    if not (isinstance(cell_integrity, np.ndarray) and cell_integrity.ndim == 2):
        raise ValueError("cell_integrity must be a 2D numpy array.")
    if not (isinstance(cell_strength, np.ndarray) and cell_strength.ndim == 2):
        raise ValueError("cell_strength must be a 2D numpy array.")

    figure_aspect_ratio = cell_integrity.shape[0] / cell_integrity.shape[1]

    with plt.style.context("dark_background"):
        # Handle colormap
        if cmap == "BIH":
            try:
                cmap = _BIH_CMAP
            except NameError:
                raise ValueError("BIH colormap is not defined.")

        # Set up subplots
        if plot_hist:
            fig, ax = plt.subplots(
                1,
                2,
                figsize=(figure_height / figure_aspect_ratio * 1.4, figure_height),
                gridspec_kw={"width_ratios": [6, 1]},
                dpi=600
            )
        else:
            fig, ax = plt.subplots(
                1, 1, figsize=(figure_height / figure_aspect_ratio, figure_height),
                dpi=600
            )
            ax = [ax]

        if plot_markers and marker_signals is not None:
            ax[0].scatter(marker_signals['x'], marker_signals['y'], s=1, c='orange', alpha=0.1)

        # Plot heatmap
        img = ax[0].imshow(
            cell_integrity,
            cmap=cmap,
            alpha=((cell_strength / signal_threshold).clip(0, 1) ** 2),
            vmin=0,
            vmax=1,
        )
        ax[0].invert_yaxis()
        # ax[0].spines[["top", "right"]].set_visible(False)
        ax[0].set_xlim(0, 1800)
        ax[0].set_ylim(0, 1800)


        # Optionally plot boundarys
        if plot_boundarys and boundary_df is not None:
            for idx, row in boundary_df.iterrows():
                ax[0].plot(row['boundaryX'], row['boundaryY'], c=boundary_color, linewidth=1.5) # for four focused regions
                # ax[0].plot(row['boundaryX'], row['boundaryY'], c=boundary_color, linewidth=0.5)
        

        # Plot histogram
        if plot_hist:
            vals, bins = np.histogram(
                cell_integrity[cell_strength > signal_threshold],
                bins=50,
                range=(0, 1),
                density=True,
            )
            colors = cmap(bins[1:-1])
            bars = ax[1].barh(bins[1:-1], vals[1:], height=0.01)
            for i, bar in enumerate(bars):
                bar.set_color(colors[i])
            ax[1].set_ylim(0, 1)
            ax[1].set_xticks([], [])
            ax[1].invert_xaxis()
            ax[1].yaxis.tick_right()
            ax[1].spines[["top", "bottom", "left"]].set_visible(False)
            ax[1].set_ylabel("signal integrity")
            ax[1].yaxis.set_label_position("right")
        # else:
        #     fig.colorbar(img, shrink=0.8)

        ax[0].set_xticks([])
        ax[0].set_yticks([])

        plt.show()

    return fig


#### overall

In [ ]:
# VSI map with cell boundaries

SIM = plot_cell_integrity(
    cell_integrity = signal_integrity,
    cell_strength = signal_strength,
    signal_threshold=3.0,
    figure_height=10,
    cmap="BIH",
    plot_hist=False,
    boundary_df=boundaries_df,
    plot_boundarys=True,
    marker_signals=None,
    plot_markers=False,
    boundary_color="yellow",
)

In [ ]:
# VSI map without cell boundaries

SIM_noboundaries = plot_cell_integrity(
    cell_integrity = signal_integrity,
    cell_strength = signal_strength,
    signal_threshold=3.0,
    figure_height=10,
    cmap="BIH",
    plot_hist=False,
    boundary_df=None,
    plot_boundarys=False,
    marker_signals=None,
    plot_markers=False,
    boundary_color="yellow",
)

#### cell level

In [ ]:
cell_integrity, cell_strength = extract_cell_integrity_within_boundary(
    boundary_df=boundaries_df, 
    sig_integrity=signal_integrity,
    signal_strength=signal_strength,
    integrity_size=1800
)

In [ ]:
cell_SIM_nb = plot_cell_integrity(
    cell_integrity = cell_integrity,
    cell_strength = cell_strength,
    signal_threshold=3.0,
    figure_height=10,
    cmap="BIH",
    # plot_hist=False,
    plot_hist=True,
    boundary_df=None,
    plot_boundarys=False,
    marker_signals=None,
    plot_markers=False,
    boundary_color="yellow",
)

In [ ]:
cell_SIM = plot_cell_integrity(
    cell_integrity = cell_integrity,
    cell_strength = cell_strength,
    signal_threshold=3.0,
    figure_height=10,
    cmap="BIH",
    plot_hist=False,
    boundary_df=boundaries_df,
    plot_boundarys=True,
    marker_signals=None,
    plot_markers=False,
    boundary_color="yellow",
)

#### ROI for findings
ROIs show the low vsi at borders and false cell identification

In [ ]:
# region 1

ax = cell_SIM.axes[0]  
ax.set_ylim(1250, 1350)
ax.set_xlim(400, 500)

cell_SIM

In [ ]:
# region 2

# Access the heatmap axis and set zoomed-in limits
ax = cell_SIM.axes[0]  
ax.set_ylim(680, 780)
ax.set_xlim(1400, 1500)

cell_SIM

In [ ]:
# region 3

# Access the heatmap axis and set zoomed-in limits
ax = cell_SIM.axes[0]  
ax.set_ylim(460, 560)
ax.set_xlim(480, 580)

cell_SIM

In [ ]:
# region 4

ax = cell_SIM.axes[0]  
ax.set_ylim(250, 350)
ax.set_xlim(1250, 1350)

cell_SIM

### VSI in MODs
compare the VSI values in MOD1 and MOD2  
- MOD2 shows lower VSI

#### functions

In [ ]:
# plot_integrity_comparison:Compare histograms and cumulative densities of two integrity.

def plot_histogram(ax, cell_integrity, cell_strength, signal_threshold, cmap, label):
    """
    Plot a histogram with color gradients based on a colormap.

    Parameters:
        ax: matplotlib axes object.
        cell_integrity: 1D array of signal integrity values.
        cell_strength: 1D array of signal strength values.
        signal_threshold: Threshold to filter cell_strength.
        cmap: Colormap for gradient coloring.
        label: Label for the x-axis.
    """
    # Calculate histogram values
    vals, bins = np.histogram(
        cell_integrity[cell_strength > signal_threshold],
        bins=50,
        range=(0, 1),
        density=True,
    )
    
    # Plot histogram
    n, bins, patches = ax.hist(
        cell_integrity[cell_strength > signal_threshold],
        bins=50,
        range=(0, 1),
        density=True,
        edgecolor='black',
        alpha=0.8
    )
    
    # Apply colormap
    for i, patch in enumerate(patches):
        patch.set_facecolor(cmap(i / len(patches)))
    
    # Customize appearance
    ax.set_xlim(0, 1)
    # ax.set_ylim(, 32)
    ax.set_yscale('log', base=2)
    ax.set_ylabel("Density")
    ax.set_xlabel(label)
    ax.spines[["top", "right"]].set_visible(False)
    ax.yaxis.set_tick_params(labelright=False)
    ax.xaxis.set_tick_params(labelsize=8)
    
    return vals, bins

def plot_cumulative_density(ax, vals1, bins1, vals2, bins2):
    """
    Plot cumulative density for two histograms.

    Parameters:
        ax: matplotlib axes object.
        vals1, bins1: Values and bins for dataset 1.
        vals2, bins2: Values and bins for dataset 2.
    """

    # Calculate cumulative densities
    cumulative1 = np.cumsum(vals1 * np.diff(bins1))
    cumulative2 = np.cumsum(vals2 * np.diff(bins2))

    # Plot cumulative density
    ax.plot(bins1[1:], cumulative1, label="OD_wm", color='blue', lw=2)
    ax.plot(bins2[1:], cumulative2, label="OD_gm", color='red', lw=2)

    # Customize appearance
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_ylabel("Cumulative Density")
    ax.set_xlabel("Signal Integrity")
    ax.legend()
    ax.spines[["top", "right"]].set_visible(False)


def plot_integrity_comparison(
    cell_integrity_1,
    cell_strength_1,
    cell_integrity_2,
    cell_strength_2,
    signal_threshold=3.0,
    figure_height=10,
    cmap="BIH",
):
    """
    Compare histograms and cumulative densities of two datasets.

    Parameters:
        cell_integrity_1, cell_strength_1: Data for dataset 1.
        cell_integrity_2, cell_strength_2: Data for dataset 2.
        signal_threshold: Threshold for filtering data.
        figure_height: Height of the figure.
        cmap: Colormap for histogram gradients.
    """
    # Validate inputs
    for data, name in [
        (cell_integrity_1, "cell_integrity_1"),
        (cell_strength_1, "cell_strength_1"),
        (cell_integrity_2, "cell_integrity_2"),
        (cell_strength_2, "cell_strength_2"),
    ]:
        if not (isinstance(data, np.ndarray) and data.ndim == 2):
            raise ValueError(f"{name} must be a 2D numpy array.")

    with plt.style.context("default"):
        # Define colormap
        if cmap == "BIH":
            try:
                cmap = _BIH_CMAP
            except NameError:
                raise ValueError("BIH colormap is not defined.")
        
        # Create figure and subplots
        fig, ax = plt.subplots(2, 1, figsize=(figure_height, figure_height), dpi=600)
            
        # Plot histograms
        vals1, bins1 = plot_histogram(
            ax[0], cell_integrity_1, cell_strength_1, signal_threshold, cmap, label="MOD1 Signal Integrity"
        )
        vals2, bins2 = plot_histogram(
            ax[1], cell_integrity_2, cell_strength_2, signal_threshold, cmap, label="MOD2 Signal Integrity"
        )

    plt.tight_layout()  # Adjust spacing to prevent overlap
    plt.show()

    return vals1, bins1, vals2, bins2

In [ ]:
# plot_qq_integrity: Create a Q-Q plot comparing two histograms with limited (0,1) range and 1:1 aspect ratio.

from scipy.interpolate import interp1d
from matplotlib.colors import Normalize

def histogram_to_cdf(vals, bins):
    """Convert histogram data to a cumulative density function (CDF)."""
    cdf = np.cumsum(vals) / np.sum(vals)  # Normalize to [0, 1]
    return bins[1:], cdf  # Return bin edges (ignoring first) and CDF

def get_quantiles(cdf_x, cdf_y, quantiles):
    """Find quantiles from histogram-based CDF."""
    interp_func = interp1d(cdf_y, cdf_x, bounds_error=False, fill_value="extrapolate")
    return interp_func(quantiles)  # Map quantiles to data values

def plot_qq_integrity(ax, vals1, bins1, vals2, bins2):
    """
    Create a Q-Q plot comparing two histograms with limited (0,1) range and 1:1 aspect ratio.

    Parameters:
        ax: Matplotlib axis object
        vals1, bins1: Histogram values and bin edges for dataset 1
        vals2, bins2: Histogram values and bin edges for dataset 2
    """
    # Convert histograms to CDFs
    x1, cdf1 = histogram_to_cdf(vals1, bins1)
    x2, cdf2 = histogram_to_cdf(vals2, bins2)

    # Define quantiles (evenly spaced percentiles)
    quantiles = np.linspace(0, 1, 100)

    # Find matching quantiles in both distributions
    q1 = get_quantiles(x1, cdf1, quantiles)
    q2 = get_quantiles(x2, cdf2, quantiles)

    # Limit data to (0,1) range
    q1 = np.clip(q1, 0, 1)
    q2 = np.clip(q2, 0, 1)

    # Plot Q-Q plot
    ax.scatter(q1, q2, color="purple", alpha=0.6, label="Q-Q Plot", s=17)
    ax.plot([0, 1], [0, 1], "k--", label="y = x")  # 1:1 reference line

    # Set axis limits and 1:1 aspect ratio
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal', adjustable='box')  # Ensure 1:1 aspect ratio

    # Customize appearance
    ax.set_xlabel("Quantiles of MOD1", fontsize=13)
    ax.set_ylabel("Quantiles of MOD2", fontsize=13) 
    # ax.set_title("Q-Q Plot (0-1 Range)")
    ax.legend(fontsize=13)
    ax.grid(False)

def plot_qq_integrity_cmap(ax, vals1, bins1, vals2, bins2, cmap="viridis"):
    """
    Create a Q-Q plot comparing two histograms with limited (0,1) range and 1:1 aspect ratio.
    
    Parameters:
        ax: Matplotlib axis object
        vals1, bins1: Histogram values and bin edges for dataset 1
        vals2, bins2: Histogram values and bin edges for dataset 2
        cmap: Colormap to apply to the Q-Q plot points (default is "viridis")
    """
    # Convert histograms to CDFs
    x1, cdf1 = histogram_to_cdf(vals1, bins1)
    x2, cdf2 = histogram_to_cdf(vals2, bins2)

    # Define quantiles (evenly spaced percentiles)
    quantiles = np.linspace(0, 1, 100)

    # Find matching quantiles in both distributions
    q1 = get_quantiles(x1, cdf1, quantiles)
    q2 = get_quantiles(x2, cdf2, quantiles)

    # Limit data to (0,1) range
    q1 = np.clip(q1, 0, 1)
    q2 = np.clip(q2, 0, 1)

    # Define colormap and normalize for the scatter plot
    norm = Normalize(vmin=0, vmax=1)  # Normalize the data to the range [0, 1]
    
    # Plot Q-Q plot with color based on quantiles
    scatter = ax.scatter(q1, q2, c=quantiles, cmap=_BIH_CMAP, alpha=0.6, s=17, norm=norm, linewidths=0.1, edgecolors='black')

    # Plot 1:1 reference line
    ax.plot([0, 1], [0, 1], "k--", label="y = x")  # 1:1 reference line

    # Add horizontal and vertical lines at 0.7 (optional)
    # ax.axhline(y=0.7, color="gray", linestyle="--", linewidth=1, label="y = 0.7")
    # ax.axvline(x=0.7, color="gray", linestyle="--", linewidth=1, label="x = 0.7")

    # Set axis limits and 1:1 aspect ratio
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal', adjustable='box')  # Ensure 1:1 aspect ratio

    # Customize appearance
    ax.set_xlabel("Quantiles of MOD1", fontsize=13)
    ax.set_ylabel("Quantiles of MOD2", fontsize=13) 
    # ax.set_title("Q-Q Plot (0-1 Range)")
    ax.legend(fontsize=13)
    ax.grid(False)



#### MOD1 vs. MOD2

In [ ]:
OD_wm_int, OD_wm_str = extract_cell_integrity_within_boundary(MOD1_boundaries, signal_integrity, signal_strength)
OD_gm_int, OD_gm_str = extract_cell_integrity_within_boundary(MOD2_boundaries, signal_integrity, signal_strength)

In [ ]:
vals1, bins1, vals2, bins2 = plot_integrity_comparison(
    cell_integrity_1=OD_wm_int,
    cell_strength_1=OD_wm_str,
    cell_integrity_2=OD_gm_int,
    cell_strength_2=OD_gm_str,
    signal_threshold=3.0,
    figure_height=7,
    cmap="BIH"
)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), dpi=600)
plot_qq_integrity(ax, vals1, bins1, vals2, bins2)

ax = plt.gca()  # Get the current axis
ax.spines['top'].set_visible(False)  # Hide the top border
ax.spines['right'].set_visible(False)  # Hide the right border
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), dpi=600)
plot_qq_integrity_cmap(ax, vals1, bins1, vals2, bins2)

ax = plt.gca()  # Get the current axis
ax.spines['top'].set_visible(False)  # Hide the top border
ax.spines['right'].set_visible(False)  # Hide the right border
plt.show()

In [ ]:
epsilon = 1e-10
vals = vals2 / (vals1 + epsilon)
bins = bins1

In [ ]:
cmap = _BIH_CMAP

bin_centers = (bins[:-1] + bins[1:]) / 2

fig, ax = plt.subplots(figsize=(6, 6), dpi=600)

# Create the histogram bars
bars = ax.bar(bin_centers, vals, width=np.diff(bins), edgecolor="black", alpha=0.7, linewidth=0.3)

# Apply colormap
for i, bar in enumerate(bars):
    bar.set_facecolor(cmap(i / len(bars)))  # Set color based on the colormap

ax.set_ylabel("VSI Density of MOD2/MOD1")
ax.set_xlabel("Signal Integrity")
ax.spines[["top", "right"]].set_visible(False)
ax.yaxis.set_tick_params(labelright=False)
ax.xaxis.set_tick_params(labelsize=8)

# Set the y-axis scale to log
ax.set_yscale('log')

# Add a horizontal dashed line at y = 1 (10^0)
ax.axhline(y=1, color='black', linestyle='--', linewidth=0.5)

plt.show()
